# Import Library & Setup

In [2]:
import os
import random
import math
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, callbacks
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization

# ML
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, calinski_harabasz_score,
    adjusted_rand_score
)
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.ensemble import RandomForestClassifier

# Viz
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

np.random.seed(42)
tf.random.set_seed(42)

print('Libraries loaded')
print(f'Time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

Libraries loaded
Time: 2026-02-10 09:01:10


# Load Data

In [3]:
file = '../../data/trail_dataset.csv'

try:
    df = pd.read_csv(file)
    print(f'Loaded: {df.shape[0]} shoes × {df.shape[1]} columns')
    display(df.head())
except FileNotFoundError:
    print(f"WARNING: '{file}' not found.")
    print("Please upload the correct dataset file to run with actual data.")

Loaded: 158 shoes × 33 columns


,brand,name,terrain_light,terrain_moderate,terrain_technical,shock_absorption,energy_return,arch_neutral,arch_stability,weight_lab_oz,...,heel_stiff,lug_dept_mm,heel_lab_mm,forefoot_lab_mm,season_summer,season_winter,season_all,removable_insole,waterproof,water_repellent
0,adidas,terrex agravic speed ultra,1,0,0,3,5,1,0,9.1,...,1,2.5,30.6,30.3,0,0,1,1,0,0
1,adidas,terrex speed ultra,1,0,0,0,0,1,0,9.1,...,1,2.6,32.8,24.6,0,0,0,1,0,0
2,altra,experience wild,1,1,0,3,1,1,0,10.1,...,3,3.6,34.5,30.2,0,0,1,1,0,0
3,altra,experience wild 2,1,0,0,3,1,1,0,9.4,...,1,3.5,32.3,26.2,0,0,1,1,0,0
4,altra,lone peak 5.0,1,1,0,0,0,1,0,10.7,...,0,3.7,24.5,24.3,0,0,0,1,0,0


In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   brand               158 non-null    str    
 1   name                158 non-null    str    
 2   terrain_light       158 non-null    int64  
 3   terrain_moderate    158 non-null    int64  
 4   terrain_technical   158 non-null    int64  
 5   shock_absorption    158 non-null    int64  
 6   energy_return       158 non-null    int64  
 7   arch_neutral        158 non-null    int64  
 8   arch_stability      158 non-null    int64  
 9   weight_lab_oz       158 non-null    float64
 10  drop_lab_mm         158 non-null    float64
 11  strike_heel         158 non-null    int64  
 12  strike_mid          158 non-null    int64  
 13  strike_forefoot     158 non-null    int64  
 14  midsole_softness    158 non-null    int64  
 15  toebox_durability   158 non-null    int64  
 16  heel_durability    

# Preprocessing

In [5]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

binary_cols = [col for col in numeric_cols if set(df[col].unique()).issubset({0, 1})]
continuous_cols = [col for col in numeric_cols if col not in binary_cols]

print(f'Features: {len(numeric_cols)} total')
print(f'  Binary     : {len(binary_cols)}')
print(f'  Continuous : {len(continuous_cols)}')

Features: 31 total
  Binary     : 16
  Continuous : 15


In [8]:
feature_cols = numeric_cols.copy()
X = df[feature_cols]

# Separate for proper scaling
X_binary = X[binary_cols].values
X_continuous = X[continuous_cols].values

# Scale continuous to 0-1 for neural network
scaler_continuous = MinMaxScaler()
X_continuous_scaled = scaler_continuous.fit_transform(X_continuous)

# Combine
X_combined = np.concatenate([X_binary, X_continuous_scaled], axis=1)

# Also standard scaling for traditional comparison
scaler_standard = StandardScaler()
X_standard = scaler_standard.fit_transform(X)

print(f'Neural input shape: {X_combined.shape}')
print(f'Range: [{X_combined.min():.6f}, {X_combined.max():.6f}]')

Neural input shape: (158, 31)
Range: [0.000000, 1.000000]


# Auto-Encoder

## Modelling

In [9]:
# Architecture
input_dim = X_combined.shape[1]
encoding_dims = [32, 16, 8]

# Encoder
input_layer = Input(shape=(input_dim,))
x = input_layer
for dim in encoding_dims:
    x = Dense(dim, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

latent = x

# Decoder
for dim in reversed(encoding_dims[:-1]):
    x = Dense(dim, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

output_layer = Dense(input_dim, activation='sigmoid')(x)

autoencoder = Model(input_layer, output_layer)
encoder = Model(input_layer, latent)

autoencoder.compile(
    optimizer=optimizers.Adam(0.001),
    loss='mse',
    metrics=['mae']
)

print('Autoencoder architecture:')
autoencoder.summary()

Autoencoder architecture:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 31)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8)              │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 31)             │         1,023 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,815 (14.90 KB)

 Trainable params: 3,607 (14.09 KB)

 Non-trainable params: 208 (832.00 B)

## Training

In [10]:
history = autoencoder.fit(
    X_combined, X_combined,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=[
        callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-5)
    ],
    verbose=0
)

print(f'Training done!')
print(f'Final loss: {history.history["loss"][-1]:.6f}')
print(f'  Val loss: {history.history["val_loss"][-1]:.6f}')

# Get latent representations
X_latent = encoder.predict(X_combined, verbose=0)
print(f'Latent space: {X_latent.shape} (8D embeddings)')

Training done!
Final loss: 0.072080
  Val loss: 0.097575
Latent space: (158, 8) (8D embeddings)


# Metrics Function

In [11]:
def calculate_cluster_purity(df, cluster_col, binary_cols):
    """
    Calculates the purity of each cluster based on binary features.
    Purity is defined as the mean dominance of the most frequent value (0 or 1)
    within each binary column for a given cluster.

    Args:
        df (pd.DataFrame): The DataFrame containing data and cluster assignments.
        cluster_col (str): The name of the column in df that contains cluster labels.
        binary_cols (list): A list of column names in df that are binary features.

    Returns:
        dict: A dictionary containing:
            - 'by_cluster': A dictionary with purity and count for each cluster.
            - 'mean_purity': The average purity across all clusters.
            - 'min_purity': The minimum purity among all clusters.
            - 'max_purity': The maximum purity among all clusters.
    """
    purity_by_cluster = {}
    for cid in df[cluster_col].unique():
        cdata = df[df[cluster_col] == cid]
        n = len(cdata)
        dominances = []
        for col in binary_cols:
            if col in cdata.columns:
                vc = cdata[col].value_counts()
                if len(vc) > 0:
                    dominances.append(vc.max() / n)
        purity_by_cluster[cid] = {'purity': np.mean(dominances) if dominances else 0, 'n': n}
    all_p = [v['purity'] for v in purity_by_cluster.values()]
    return {
        'by_cluster': purity_by_cluster,
        'mean_purity': np.mean(all_p),
        'min_purity': np.min(all_p),
        'max_purity': np.max(all_p)
    }

def calculate_cluster_stability(X, labels, model_func, n_iter=20):
    """
    Calculates the stability of clustering using the Adjusted Rand Index (ARI).
    It performs bootstrapping by re-sampling the data and re-clustering to measure
    how consistent the cluster assignments are.

    Args:
        X (np.ndarray): The feature matrix used for clustering.
        labels (np.ndarray): The original cluster labels from the initial clustering.
        model_func (callable): A function that returns a new, untrained clustering model
                                (e.g., `lambda: KMeans(n_clusters=k)`).
        n_iter (int, optional): The number of bootstrap iterations. Defaults to 20.

    Returns:
        dict: A dictionary containing:
            - 'mean_ari': The mean Adjusted Rand Index.
            - 'std_ari': The standard deviation of the ARI scores.
            - 'stability_level': A categorical label (Excellent, Good, Moderate)
                                 based on the mean ARI.
    """
    n = len(X)
    ari_scores = []
    for _ in range(n_iter):
        idx = np.random.choice(n, n, replace=True)
        # Ensure model_func returns a new, untrained model each time
        boot_model = model_func()
        boot_labels = boot_model.fit_predict(X[idx])
        ari = adjusted_rand_score(labels[idx], boot_labels)
        ari_scores.append(ari)
    m = np.mean(ari_scores)
    return {
        'mean_ari': m,
        'std_ari': np.std(ari_scores),
        'stability_level': 'Excellent' if m > 0.8 else 'Good' if m > 0.6 else 'Moderate'
    }

def calculate_interpretability_score(df, cluster_col, binary_cols, threshold=0.75):
    """
    Calculates an interpretability score for each cluster.
    A cluster is considered more interpretable if a high proportion of its members
    strongly exhibit (or strongly do not exhibit) certain binary features.

    Args:
        df (pd.DataFrame): The DataFrame containing data and cluster assignments.
        cluster_col (str): The name of the column in df that contains cluster labels.
        binary_cols (list): A list of column names in df that are binary features.
        threshold (float, optional): The threshold for defining strong exhibition.
                                     A feature is 'strong' if its mean in a cluster
                                     is > threshold or < (1 - threshold). Defaults to 0.75.

    Returns:
        dict: A dictionary containing:
            - 'mean_interpretability': The average interpretability score across all clusters.
            - 'scores': A list of interpretability scores for each cluster.
    """
    scores = []
    for cid in df[cluster_col].unique():
        cdata = df[df[cluster_col] == cid]
        n = len(cdata)
        strong = sum(1 for col in binary_cols if col in cdata.columns and
                    (cdata[col].sum()/n > threshold or cdata[col].sum()/n < 1-threshold))
        # Score is the proportion of binary features that are 'strong' for this cluster
        scores.append(strong / len(binary_cols))
    return {'mean_interpretability': np.mean(scores), 'scores': scores}

def evaluate_clustering_comprehensive(X, labels, df_temp, model_func, binary_cols):
    """
    Performs a comprehensive evaluation of clustering results using multiple metrics.
    It calculates Silhouette, Davies-Bouldin, Calinski-Harabasz scores, as well as
    custom purity, stability, and interpretability scores.
    A composite score is then calculated based on a weighted average of normalized metrics.

    Args:
        X (np.ndarray): The feature matrix used for clustering.
        labels (np.ndarray): The cluster labels generated by the clustering algorithm.
        df_temp (pd.DataFrame): A temporary DataFrame, copy of the original, to add cluster labels.
        model_func (callable): A function that returns a new, untrained clustering model
                                (used for stability calculation).
        binary_cols (list): A list of column names in df_temp that are binary features.

    Returns:
        dict: A dictionary containing various evaluation metrics and a composite score:
            - 'silhouette': Silhouette Score.
            - 'davies_bouldin': Davies-Bouldin Score.
            - 'calinski_harabasz': Calinski-Harabasz Score.
            - 'purity': Mean cluster purity.
            - 'stability': Mean Adjusted Rand Index from stability testing.
            - 'interpretability': Mean cluster interpretability score.
            - 'composite_score': A weighted composite score of normalized metrics.
    """
    sil = silhouette_score(X, labels)
    db = davies_bouldin_score(X, labels)
    ch = calinski_harabasz_score(X, labels)
    df_temp['cluster'] = labels
    purity = calculate_cluster_purity(df_temp, 'cluster', binary_cols)
    stability = calculate_cluster_stability(X, labels, model_func, 10)
    interp = calculate_interpretability_score(df_temp, 'cluster', binary_cols)

    # Normalize scores for composite calculation
    sil_norm = (sil + 1) / 2
    db_norm = 1 / (1 + db)
    ch_norm = min(ch / 1000, 1)

    # Composite score with example weights
    composite = (0.25*sil_norm + 0.20*db_norm + 0.15*ch_norm +
                 0.25*purity['mean_purity'] + 0.10*stability['mean_ari'] +
                 0.05*interp['mean_interpretability'])

    return {
        'silhouette': sil, 'davies_bouldin': db, 'calinski_harabasz': ch,
        'purity': purity['mean_purity'], 'stability': stability['mean_ari'],
        'interpretability': interp['mean_interpretability'], 'composite_score': composite
    }

print('Metrics functions ready')

Metrics functions ready


# Model Selection

In [12]:
results = []

# Header Tabel
print(f"| {'K':^3} | {'Score':^8} | {'Sil.':^8} | {'DB':^8} | {'CH':^10} | {'Purity':^8} | {'Stab.':^8} | {'Interp':^8} |")
print(f"|{'-'*5}+{'-'*10}+{'-'*10}+{'-'*10}+{'-'*12}+{'-'*10}+{'-'*10}+{'-'*10}|")

for i in range(3, 10):
    np.random.seed(42)
    
    model_factory = lambda: KMeans(n_clusters=i, random_state=42, n_init=10)
    model = model_factory()
    labels = model.fit_predict(X_latent)

    metrics = evaluate_clustering_comprehensive(
        X_latent, labels, df.copy(),
        model_factory,
        binary_cols
    )

    # Simpan hasil
    results.append({
        'k': i,
        'model': model,
        'labels': labels,
        **metrics
    })

    # Print Baris Tabel
    print(f"| {i:^3} | {metrics['composite_score']:<8.6f} | {metrics['silhouette']:<6.6f} | "
          f"{metrics['davies_bouldin']:<6.6f} | {metrics['calinski_harabasz']:<8.6f} | "
          f"{metrics['purity']:<6.6f} | {metrics['stability']:<6.6f} | {metrics['interpretability']:<6.6f} |")

|  K  |  Score   |   Sil.   |    DB    |     CH     |  Purity  |  Stab.   |  Interp  |
|-----+----------+----------+----------+------------+----------+----------+----------|
|  3  | 0.666632 | 0.494526 | 0.833814 | 120.914306 | 0.865731 | 0.966011 | 0.791667 |
|  4  | 0.672299 | 0.490555 | 0.760047 | 135.967871 | 0.863977 | 0.953318 | 0.812500 |
|  5  | 0.635842 | 0.367890 | 0.929490 | 126.544045 | 0.878209 | 0.820421 | 0.812500 |
|  6  | 0.640412 | 0.367252 | 0.925535 | 119.559908 | 0.894479 | 0.818965 | 0.843750 |
|  7  | 0.630861 | 0.377165 | 1.012078 | 117.873501 | 0.887777 | 0.777265 | 0.839286 |
|  8  | 0.629461 | 0.369986 | 0.999311 | 116.258834 | 0.897214 | 0.765927 | 0.796875 |
|  9  | 0.640633 | 0.374334 | 0.978029 | 116.685722 | 0.899664 | 0.843398 | 0.819444 |


In [13]:
df_results = pd.DataFrame(results)
best_config = df_results.loc[df_results['composite_score'].idxmax()]

best_model = best_config['model']
best_labels = best_config['labels']
best_k = best_config['k']
X_for_clustering = X_latent

print(f'SELECTED BEST K: {best_k}')
print(f'   Silhouette      : {best_config["silhouette"]:.6f}')
print(f'   Composite Score : {best_config["composite_score"]:.6f}')

SELECTED BEST K: 4
   Silhouette      : 0.490555
   Composite Score : 0.672299


# Binning

In [14]:
for col in df.select_dtypes('float64').columns.tolist():
    new_col_name = col + '_bin'
    df[new_col_name] = pd.qcut(df[col], q=3, labels=[0, 0.5, 1]).astype(int)

# Reorder columns: non-numeric, binary, then continuous with their bins, then cluster
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

new_column_order = []

# non-numeric columns
for col in non_numeric_cols:
    if col in df.columns:
        new_column_order.append(col)
# binary columns
for col in binary_cols:
    if col in df.columns:
        new_column_order.append(col)
# continuous columns and their corresponding bin columns
for col in continuous_cols:
    if col in df.columns:
        new_column_order.append(col)
    bin_col_name = col + '_bin'
    if bin_col_name in df.columns:
        new_column_order.append(bin_col_name)

# Add the 'cluster' column
if 'cluster' in df.columns and 'cluster' not in new_column_order:
    new_column_order.append('cluster')

# Reindex the DataFrame with the new order
df = df[new_column_order]

In [15]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   brand                158 non-null    str    
 1   name                 158 non-null    str    
 2   terrain_light        158 non-null    int64  
 3   terrain_moderate     158 non-null    int64  
 4   terrain_technical    158 non-null    int64  
 5   arch_neutral         158 non-null    int64  
 6   arch_stability       158 non-null    int64  
 7   strike_heel          158 non-null    int64  
 8   strike_mid           158 non-null    int64  
 9   strike_forefoot      158 non-null    int64  
 10  plate_rock_plate     158 non-null    int64  
 11  plate_carbon_plate   158 non-null    int64  
 12  season_summer        158 non-null    int64  
 13  season_winter        158 non-null    int64  
 14  season_all           158 non-null    int64  
 15  removable_insole     158 non-null    int64  
 16  w

# Generate Cluster Labels

In [16]:
# Masukkan Cluster ke DataFrame
df['cluster'] = best_labels 

# Setup Grouping
bin_groups = {}
for col in binary_cols:
    parts = col.split('_')
    
    if len(parts) > 1:
        prefix = '_'.join(parts[:-1])
    else:
        prefix = col
        
    bin_groups.setdefault(prefix, []).append(col)

# Build Summary Data
rows = []
for cid in sorted(df['cluster'].unique()):
    subset = df[df['cluster'] == cid]
    n = len(subset)
    
    row = {'count': n, 'percentage': f"{n/len(df)*100:.1f}%"}

    # A. Continuous Columns: Langsung ambil mean
    for col in continuous_cols:
        row[col.lower()] = round(subset[col].mean(), 2)

    # B. Binary Groups
    for prefix, cols in bin_groups.items():
        # Hitung mean grup ini
        means = subset[cols].mean()
        best_col = means.idxmax()
        best_val = means.max()
        
        # Case 1: Multiple Variants
        if len(cols) > 1:
            header = prefix.lower()
            val_str = best_col.replace(f"{prefix}_", "").lower()
            row[header] = f"{val_str} ({best_val*100:.0f}%)"
            
        # Case 2: Standalone
        else:
            header = cols[0].lower()
            val_str = "yes" if best_val > 0.5 else "no"
            row[header] = f"{val_str} ({best_val*100:.0f}%)"

    rows.append(row)

# Create DataFrame & Fix Display
df_summary = pd.DataFrame(rows, index=sorted(df['cluster'].unique()))
df_summary.index.name = None 

print("Cluster Summary:")
display(df_summary)

Cluster Summary:


,count,percentage,shock_absorption,energy_return,weight_lab_oz,drop_lab_mm,midsole_softness,toebox_durability,heel_durability,outsole_durability,...,forefoot_lab_mm,terrain,arch,strike,plate_rock_plate,plate_carbon_plate,season,removable_insole,waterproof,water_repellent
0,91,57.6%,1.91,1.43,10.09,5.74,3.74,2.92,2.98,3.24,...,25.97,light (69%),neutral (100%),mid (100%),no (20%),no (4%),all (90%),yes (95%),no (7%),no (4%)
1,34,21.5%,1.97,1.09,10.30,12.03,3.41,3.26,3.32,3.68,...,24.55,light (79%),neutral (94%),heel (100%),no (18%),no (12%),all (79%),yes (100%),no (18%),no (3%)
2,18,11.4%,0.00,0.00,10.09,4.52,0.89,0.00,0.00,0.00,...,24.52,moderate (78%),neutral (100%),mid (100%),no (50%),no (6%),all (33%),yes (67%),no (0%),no (6%)
3,15,9.5%,0.40,0.13,10.70,11.11,1.80,0.33,0.47,0.27,...,24.06,light (60%),neutral (87%),heel (93%),no (13%),no (13%),all (53%),yes (100%),no (7%),no (0%)
